In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = (SparkSession.builder
            .appName("StoreAnalysis")
            .master("local[*]")  # Use local Spark execution with all available cores
            .config("spark.jars.packages",
                    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")  # Spark-Kafka integration
            .config("spark.jars",
                    "postgresql-42.7.1.jar")  # PostgreSQL driver
            .config("spark.sql.adaptive.enabled", "false")  # Disable adaptive query execution
            .getOrCreate())

In [3]:
df = spark.read.option("header", True).csv('sample_superstore.csv')
df.show(5,False)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+-----------------------------------------------------------+--------+--------+--------+--------+
|row_id|order_id      |order_date|ship_date |ship_mode     |customer_id|customer_name  |segment  |country      |city           |state     |postal_code|region|product_id     |category       |sub_category|product_name                                               |sales   |quantity|discount|profit  |
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+-----------------------------------------------------------+--------+--------+--------+--------+
|1     |CA-2016-152156|11/8/2016 |11/11/2016|Second Class  |CG-12520   |Claire Gute    |Consumer |Un

In [4]:
store_schema = StructType([
        StructField("ts_id", StringType(), True),
        StructField("ts", StringType(), True),
        StructField("customer_id", StringType(), True),
        StructField("customer_name", StringType(), True),
        StructField("segment", StringType(), True),
        StructField("country", StringType(), True),
        StructField("city", StringType(), True),
        StructField("category", StringType(), True),
        StructField("sub_category", StringType(), True),
        StructField("product_name", StringType(), True),
        StructField("price", StringType(), True),
        StructField("quantity", StringType(), True),
        StructField("discount", StringType(), True),
        StructField("revenue", StringType(), True),
        StructField("lat_long", StringType(), True)
])

In [5]:
store_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:9092") \
    .option("subscribe", "store") \
    .option("startingOffsets", "earliest") \
    .load()

    # .selectExpr("CAST(value AS STRING)") \
    # .select(from_json(col("value"), store_schema).alias("data")) \
    # .select("data.*")

In [6]:
store_df

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]